In [0]:
#Eliminacion de datos residuales en databricks comunity edition
dbutils.fs.rm('dbfs:/user/hive/warehouse/week',recurse=True)

Out[1]: True

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/dim_time',recurse=True)

Out[2]: True

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/dim_pizzas',recurse=True)

Out[4]: True

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/fact_orders',recurse=True)

Out[3]: True

#Catalogo de Datos

Table	| Field	|Description
----------|------------|---------------
orders	|order_id|	Unique identifier for each order placed by a table
orders|	date|	Date the order was placed (entered into the system prior to cooking & serving)
orders	|time|	Time the order was placed (entered into the system prior to cooking & serving)
order_details|	order_details_id|	Unique identifier for each pizza placed within each order (pizzas of the same type and size are kept in the same row, and the quantity increases)
order_details|	order_id|	Foreign key that ties the details in each order to the order itself
order_details|	pizza_id|	Foreign key that ties the pizza ordered to its details, like size and price
order_details|	quantity|	Quantity ordered for each pizza of the same type and size
pizzas|	pizza_id|	Unique identifier for each pizza (constituted by its type and size)
pizzas|	pizza_type_id|	Foreign key that ties each pizza to its broader pizza type
pizzas|	size|	Size of the pizza (Small, Medium, Large, X Large, or XX Large)
pizzas|	price|	Price of the pizza in USD
pizza_types|	pizza_type_id|	Unique identifier for each pizza type
pizza_types|	name|	Name of the pizza as shown in the menu
pizza_types|	category|	Category that the pizza fall under in the menu (Classic, Chicken, Supreme, or Veggie)
pizza_types|	ingredients|	Comma-delimited ingredients used in the pizza as shown in the menu (they all include Mozzarella Cheese, even if not specified

## Diagrama E-R de las tablas operativas

![E-R tablas pizza_challenge](files/imagenes/pizza_challenge_diagram_E_R_2_drawio.png)

#Identificamos Hechos y Dimensiones

* __Fact : orders__
* __Dims : time
           , pizzas__

##Modelamos la dim pizzas

Conformemos nuestra dim pizzas de las tablas pizzas, pizza_types, desnormalizando y uniendo las mismas (el precio irá en la tabla de hechos).

In [0]:
                                                                             %sql
CREATE OR REPLACE TABLE dim_pizzas_temporal AS
  SELECT
            pzz.pizza_id,
            pzz_t.pizza_type_id,
            pzz.size,
            pzz.price,
            pzz_t.name,
            pzz_t.category,
            pzz_t.ingredients

  FROM      pizzas         AS    pzz 

  LEFT JOIN pizza_types    AS    pzz_t

  ON        pzz.pizza_type_id = pzz_t.pizza_type_id

  ORDER BY pizza_id ASC

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM dim_pizzas_temporal

pizza_id,pizza_type_id,size,price,name,category,ingredients
bbq_ckn_l,bbq_ckn,L,20.75,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
bbq_ckn_m,bbq_ckn,M,16.75,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
bbq_ckn_s,bbq_ckn,S,12.75,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
big_meat_l,big_meat,L,20.5,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
big_meat_m,big_meat,M,16.0,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
big_meat_s,big_meat,S,12.0,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sausage"
brie_carre_s,brie_carre,S,23.65,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Onions, Pears, Thyme, Garlic"
calabrese_l,calabrese,L,20.25,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
calabrese_m,calabrese,M,16.25,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"
calabrese_s,calabrese,S,12.25,The Calabrese Pizza,Supreme,"�Nduja Salami, Pancetta, Tomatoes, Red Onions, Friggitello Peppers, Garlic"


Crearemos una clave subrogada incremental, para unirla con la tabla de hechos.

##Modelamos la dim tiempo

In [0]:
%sql
-- Creamos la tabla con los días de la semana
CREATE OR REPLACE TABLE week(
  id    INT,
  day   STRING
);

In [0]:
%sql
-- insertamos los días con sus id's
INSERT INTO week VALUES (1,'Lunes');
INSERT INTO week VALUES (2,'Martes');
INSERT INTO week VALUES (3,'Miercoles');
INSERT INTO week VALUES (4,'Jueves');
INSERT INTO week VALUES (5,'Viernes');
INSERT INTO week VALUES (6,'Sabado');
INSERT INTO week VALUES (7,'Domingo')

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT * FROM week ORDER BY id

id,day
1,Lunes
2,Martes
3,Miercoles
4,Jueves
5,Viernes
6,Sabado
7,Domingo


In [0]:
%sql
--Creamos la tabla con los meses de año
CREATE OR REPLACE TABLE months(
  id      INT,
  month   STRING
);

In [0]:

%sql
-- insertamos los meses con sus id's
INSERT INTO months VALUES (1,'Enero');
INSERT INTO months VALUES (2,'Febrero');
INSERT INTO months VALUES (3,'Marzo');
INSERT INTO months VALUES (4,'Abril');
INSERT INTO months VALUES (5,'Mayo');
INSERT INTO months VALUES (6,'Junio');
INSERT INTO months VALUES (7,'Julio');
INSERT INTO months VALUES (8,'Agosto');
INSERT INTO months VALUES (9,'Septiembre');
INSERT INTO months VALUES (10,'Octubre');
INSERT INTO months VALUES (11,'Noviembre');
INSERT INTO months VALUES (12,'Diciembre');

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
-- INTENTO QUEDARME A Lo SUMO CON SOLO 24hs (o las horas que registren pedidos) por cada fecha unica registrada . 
CREATE OR REPLACE TABLE dim_time_temporal AS
  SELECT DISTINCT  
                  date,              
                  extract(YEAR FROM date)  AS year,
                  extract(MONTH FROM date) AS month ,
                  extract(DAY FROM date)   AS day,
                  extract(HOUR FROM time)  AS hour, 
                  DAYOFWEEK(date)          AS day_week

  FROM            orders 

  ORDER BY       (date,hour)

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM dim_time_temporal

date,year,month,day,hour,day_week
2015-01-01,2015,1,1,11,5
2015-01-01,2015,1,1,12,5
2015-01-01,2015,1,1,13,5
2015-01-01,2015,1,1,14,5
2015-01-01,2015,1,1,15,5
2015-01-01,2015,1,1,16,5
2015-01-01,2015,1,1,17,5
2015-01-01,2015,1,1,18,5
2015-01-01,2015,1,1,19,5
2015-01-01,2015,1,1,20,5


In [0]:
%sql
-- Etiquetamos los nombres de la semana
CREATE OR REPLACE TABLE dim_time_temporal AS
    SELECT 
            time.date,
            time.year,
            time.month,
            time.day,
            time.hour,
            wk.day                  AS day_week


      FROM      dim_time_temporal   AS time

      LEFT JOIN week                AS wk

      ON        time.day_week = wk.id

num_affected_rows,num_inserted_rows


date,year,month,day,hour,day_week
2015-01-01,2015,1,1,11,Viernes
2015-01-01,2015,1,1,12,Viernes
2015-01-01,2015,1,1,13,Viernes
2015-01-01,2015,1,1,14,Viernes
2015-01-01,2015,1,1,15,Viernes
2015-01-01,2015,1,1,16,Viernes
2015-01-01,2015,1,1,17,Viernes
2015-01-01,2015,1,1,18,Viernes
2015-01-01,2015,1,1,19,Viernes
2015-01-01,2015,1,1,20,Viernes


col_name,data_type,comment
date,date,null
year,int,null
month,int,null
day,int,null
hour,int,null
day_week,string,null


In [0]:
%sql
SELECT * FROM dim_time_temporal LIMIT (1)

date,year,month,day,hour,day_week
2015-01-01,2015,1,1,11,Viernes


In [0]:
%sql
-- Etiquetamos los nombres de los meses
CREATE OR REPLACE TABLE dim_time_temporal AS
  SELECT 
        time.date,
        time.year,
        mths.month              AS month,
        time.day,
        time.hour,
        time.day_week                  


  FROM      dim_time_temporal   AS time

  LEFT JOIN months              AS mths

  ON        time.month = mths.id

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM dim_time_temporal

date,year,month,day,hour,day_week
2015-01-01,2015,Enero,1,11,Viernes
2015-01-01,2015,Enero,1,12,Viernes
2015-01-01,2015,Enero,1,13,Viernes
2015-01-01,2015,Enero,1,14,Viernes
2015-01-01,2015,Enero,1,15,Viernes
2015-01-01,2015,Enero,1,16,Viernes
2015-01-01,2015,Enero,1,17,Viernes
2015-01-01,2015,Enero,1,18,Viernes
2015-01-01,2015,Enero,1,19,Viernes
2015-01-01,2015,Enero,1,20,Viernes


Crearemos una clave subrogada incremental, para unirla con la tabla de hechos.

##Modelamos la fact ordenes

Crearemos la tabla de hechos, combinando order_details y orders, y relacionandonalo con las dimensiones con claves foraneas (haciendo uso de las claves subrogadas de las tablas. Ahorraremos espacio en nuestra fact a toda costa).

In [0]:
%sql
CREATE OR REPLACE TABLE fact_orders_temporal AS
  SELECT
        ords.order_id,
        ords.date,
        date_format(ords.time,'HH:mm:ss') as time,
        ords_dts.order_details_id,
        ords_dts.pizza_id,
        ords_dts.quantity

  FROM orders                AS    ords

  LEFT JOIN order_details    AS    ords_dts

  ON        ords_dts.order_id = ords.order_id



num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM fact_orders_temporal LIMIT(10)

order_id,date,time,order_details_id,pizza_id,quantity
1,2015-01-01,11:38:36,1,hawaiian_m,1
2,2015-01-01,11:57:40,6,thai_ckn_l,1
2,2015-01-01,11:57:40,5,mexicana_m,1
2,2015-01-01,11:57:40,4,ital_supr_l,1
2,2015-01-01,11:57:40,3,five_cheese_l,1
2,2015-01-01,11:57:40,2,classic_dlx_m,1
3,2015-01-01,12:12:28,8,prsc_argla_l,1
3,2015-01-01,12:12:28,7,ital_supr_m,1
4,2015-01-01,12:16:31,9,ital_supr_m,1
5,2015-01-01,12:21:30,10,ital_supr_m,1


In [0]:
%sql
-- Cruzamos con la tabla pizzas para calcular el precio del registro en la tabla orders (quantity * price)
CREATE OR REPLACE TABLE fact_orders_temporal AS
  SELECT
        ords.order_id,
        ords.date,
        ords.time,
        ords.order_details_id,
        ords.pizza_id,
        ords.quantity,
        (ords.quantity * pzz.price) AS price

  FROM  fact_orders_temporal        AS ords

  LEFT JOIN dim_pizzas_temporal     AS pzz

  ON    ords.pizza_id = pzz.pizza_id



num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM fact_orders_temporal

order_id,date,time,order_details_id,pizza_id,quantity,price
1,2015-01-01,11:38:36,1,hawaiian_m,1,13.25
2,2015-01-01,11:57:40,6,thai_ckn_l,1,20.75
2,2015-01-01,11:57:40,5,mexicana_m,1,16.0
2,2015-01-01,11:57:40,4,ital_supr_l,1,20.75
2,2015-01-01,11:57:40,3,five_cheese_l,1,18.5
2,2015-01-01,11:57:40,2,classic_dlx_m,1,16.0
3,2015-01-01,12:12:28,8,prsc_argla_l,1,20.75
3,2015-01-01,12:12:28,7,ital_supr_m,1,16.5
4,2015-01-01,12:16:31,9,ital_supr_m,1,16.5
5,2015-01-01,12:21:30,10,ital_supr_m,1,16.5


## Creamos la tabla dim tiempo (dim_time)

In [0]:
%sql
DESCRIBE TABLE dim_time_temporal

col_name,data_type,comment
date,date,null
year,int,null
month,string,null
day,int,null
hour,int,null
day_week,string,null


In [0]:
%sql
--CREAMOS LA TABLA DIMENSION TIEMPO VACIA (dim_time)

CREATE TABLE  dim_time(
                      id BIGINT GENERATED BY DEFAULT AS IDENTITY,
                      date      DATE,
                      year      INT,
                      month     STRING,
                      day       INT,
                      hour      INT,
                      day_week  STRING
);

In [0]:
%sql
--INSERTAMOS VALORES EN LA DIMENSION TIEMPO PARA GENERAR LA CLAVE SUBROGADA (incremental) POR CADA REGISTRO
--https://www.databricks.com/blog/2022/08/08/identity-columns-to-generate-surrogate-keys-are-now-available-in-a-lakehouse-near-you.html
-- https://community.databricks.com/s/question/0D58Y00009COZfHSAX/how-to-add-an-identity-column-to-an-existing-table

INSERT INTO dim_time (
                 date,
                 year,
                 month,
                 day,
                 hour, 
                 day_week
)
                     
SELECT TRY_CAST( date     AS DATE),
       TRY_CAST( year     AS INT),
       TRY_CAST( month    AS STRING),
       TRY_CAST( day      AS INT),
       TRY_CAST( hour     AS INT),
       TRY_CAST( day_week AS STRING)
       
FROM dim_time_temporal;

num_affected_rows,num_inserted_rows
4181,4181


## Creamos la tabla dim pizzas (dim_pizzas)

In [0]:
%sql
DESCRIBE TABLE dim_pizzas_temporal

col_name,data_type,comment
pizza_id,string,null
pizza_type_id,string,null
size,string,null
price,double,null
name,string,null
category,string,null
ingredients,string,null


In [0]:
%sql
--CREAMOS LA TABLA DIMENSION PIZZAS VACIA (dim_pizzas)
CREATE TABLE  dim_pizzas(
                      id BIGINT GENERATED BY DEFAULT AS IDENTITY,
                      pizza_id           STRING,
                      pizza_type_id      STRING,
                      size               STRING,
                      price              DOUBLE,
                      name               STRING,
                      category           STRING,
                      ingredients        STRING
);

In [0]:
%sql
INSERT INTO dim_pizzas (
                 pizza_id,           
                 pizza_type_id,     
                 size,               
                 price,              
                 name,               
                 category,           
                 ingredients       
)
                     
SELECT TRY_CAST( pizza_id          AS STRING),
       TRY_CAST( pizza_type_id     AS STRING),
       TRY_CAST( size              AS STRING),
       TRY_CAST( price             AS DOUBLE),
       TRY_CAST( name              AS STRING),
       TRY_CAST( category          AS STRING),
       TRY_CAST( ingredients       AS STRING)
       
FROM dim_pizzas_temporal;

num_affected_rows,num_inserted_rows
96,96


## Creamos la tabla fact orders (fact_orders)

In [0]:
%sql
DESCRIBE TABLE fact_orders_temporal

col_name,data_type,comment
order_id,int,null
date,date,null
time,string,null
order_details_id,int,null
pizza_id,string,null
quantity,int,null
price,double,null


In [0]:
%sql
DESCRIBE TABLE dim_time

col_name,data_type,comment
id,bigint,null
date,date,null
year,int,null
month,string,null
day,int,null
hour,int,null
day_week,string,null


Cruzamos con la tabla tiempo para relacionar la tabla de hechos solo por clave subrogada de esta (columna id de la tabla time)

In [0]:
%sql
CREATE OR REPLACE TABLE fact_orders_temporal AS
    SELECT
        ords.order_id,
        dim_t.id                    AS    dim_time_id,
        ords.order_details_id,
        ords.pizza_id,
        ords.quantity,
        ords.price

    FROM      fact_orders_temporal  AS    ords

    LEFT JOIN dim_time              AS    dim_t

    ON        ords.date = dim_t.date     AND   extract(HOUR FROM time) = dim_t.hour 

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE TABLE fact_orders_temporal

col_name,data_type,comment
order_id,int,null
dim_time_id,bigint,null
order_details_id,int,null
pizza_id,string,null
quantity,int,null
price,double,null


In [0]:
%sql
DESCRIBE TABLE dim_pizzas

col_name,data_type,comment
id,bigint,null
pizza_id,string,null
pizza_type_id,string,null
size,string,null
price,double,null
name,string,null
category,string,null
ingredients,string,null


Cruzamos con la tabla pizzas para relacionar la tabla de hechos solo por clave subrogada de esta (columna id de la tabla pizzas)

In [0]:
%sql
CREATE OR REPLACE TABLE fact_orders_temporal
    SELECT
              ords.order_id,
              ords.dim_time_id,
              dim_p.id                    AS    dim_pizzas_id,
              ords.quantity,
              ords.price

    FROM      fact_orders_temporal        AS    ords

    LEFT JOIN dim_pizzas                  AS    dim_p

    ON        ords.pizza_id = dim_p.pizza_id

num_affected_rows,num_inserted_rows


In [0]:
%sql
--CREAMOS LA TABLA DE HECHOS ORDENES VACIA (fact_orders)
CREATE TABLE  fact_orders(
                      id BIGINT GENERATED BY DEFAULT AS IDENTITY,
                      order_id             INT,
                      dim_time_id          BIGINT,
                      dim_pizzas_id        BIGINT,
                      quantity             INT,
                      price                DOUBLE
);

In [0]:
%sql
INSERT INTO fact_orders (
                order_id,           
                dim_time_id,          
                dim_pizzas_id,        
                quantity,           
                price                      
)
                     
SELECT TRY_CAST( order_id           AS    INT),
       TRY_CAST( dim_time_id        AS    BIGINT),
       TRY_CAST( dim_pizzas_id      AS    BIGINT),
       TRY_CAST( quantity           AS    INT),
       TRY_CAST( price              AS    DOUBLE)
       
FROM fact_orders_temporal;

num_affected_rows,num_inserted_rows
48620,48620


In [0]:
%sql
DESCRIBE fact_orders

col_name,data_type,comment
id,bigint,null
order_id,int,null
dim_time_id,bigint,null
dim_pizzas_id,bigint,null
quantity,int,null
price,double,null


In [0]:
%sql
DESCRIBE dim_time

col_name,data_type,comment
id,bigint,null
date,date,null
year,int,null
month,string,null
day,int,null
hour,int,null
day_week,string,null


In [0]:
%sql
DESCRIBE dim_pizzas

col_name,data_type,comment
id,bigint,null
pizza_id,string,null
pizza_type_id,string,null
size,string,null
price,double,null
name,string,null
category,string,null
ingredients,string,null


#Modelo Estrella

Aquí debí crear el modelo estrella con la misma herramienta que diagrame las tablas del comienzo, pero dejo el que muestra qlik sense, luego lo cambio

![E-R tablas pizza_challenge](files/imagenes/star_schema.PNG)